# Random Forest Classifier for Diabetes Data

**Buckley Dowdle, Latifa Hasan, Luke Moles, Jae Sung**

In [14]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

In [15]:
spark = SparkSession.builder \
    .master('local') \
    .appName('projectModeling') \
    .getOrCreate()

In [16]:
#build evaluators
f1_evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
accuracy_evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')

In [17]:
#read in data
data = spark.read.parquet("data.parquet")

feats = [
       'Gender',
       'Age',
       'Race_vector',
       'Fam_hist',
       'Smoke_Cigs',
       'BMI',
        'HighBP' 
       ]

data.count()

27297

In [18]:
#check data
data.take(5)

[Row(ParticipantID=94054.0, label=1.0, Gender=0.0, Age=50.0, Race=1.0, Fam_hist=0.0, Smoke_Cigs=3.0, BMI=30.4, HighBP=1.0),
 Row(ParticipantID=94285.0, label=1.0, Gender=0.0, Age=43.0, Race=2.0, Fam_hist=0.0, Smoke_Cigs=1.0, BMI=43.7, HighBP=1.0),
 Row(ParticipantID=94285.0, label=1.0, Gender=0.0, Age=43.0, Race=2.0, Fam_hist=0.0, Smoke_Cigs=1.0, BMI=43.7, HighBP=1.0),
 Row(ParticipantID=94285.0, label=1.0, Gender=0.0, Age=43.0, Race=2.0, Fam_hist=0.0, Smoke_Cigs=1.0, BMI=43.7, HighBP=1.0),
 Row(ParticipantID=94285.0, label=1.0, Gender=0.0, Age=43.0, Race=2.0, Fam_hist=0.0, Smoke_Cigs=1.0, BMI=43.7, HighBP=1.0)]

In [19]:
# test-train split
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [20]:
#one hot encode Race
encoder = OneHotEncoder(inputCol="Race", outputCol="Race_vector")

#make features into vector
assembler = VectorAssembler(inputCols=feats, outputCol='features')

#logistic regression model
lr = LogisticRegression(maxIter=10)

#model2
#model2 = 

#model3
#model3 = 

#create pipelines
lr_pipeline = Pipeline(stages=[encoder, assembler, lr])


#pipeline2 = Pipeline(stages=[encoder, assembler, model2])


#pipeline3 = Pipeline(stages=[encoder, assembler, model3])



In [21]:
# Set up the parameter grid
lr_paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

# Treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
lr_crossval = CrossValidator(estimator=lr_pipeline,
                          estimatorParamMaps=lr_paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName='f1'),
                          numFolds=10,
                          seed=314,
                          parallelism=7)
#fit lr_model
lr_model = lr_crossval.fit(trainingData)

#make predictions using lr
lr_preds = lr_model.transform(testData)


In [22]:
#test performance
lr_f1 = f1_evaluator.evaluate(lr_preds)
lr_acc = accuracy_evaluator.evaluate(lr_preds)

print('logistic Regression F1 Score: {}'.format(lr_f1))
print('logistic Regression Accuracy: {}'.format(lr_acc))

logistic Regression F1 Score: 0.770537335790141
logistic Regression Accuracy: 0.8082307506637703


In [ ]:
# Set up the parameter grid
model2_paramGrid = ParamGridBuilder() \
    .addGrid() \
    .build()

# Treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
model2_crossval = CrossValidator(estimator=model2_pipeline,
                          estimatorParamMaps=model2_paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName='f1'),
                          numFolds=10,
                          seed=314,
                          parallelism=7)
#fit lr_model
model2_model = model2_crossval.fit(trainingData)

#make predictions using lr
model2_preds = model2_model.transform(testData)

In [ ]:
#test performance
model2_f1 = f1_evaluator.evaluate(model2_preds)
model2_acc = accuracy_evaluator.evaluate(model2_preds)

print('logistic Regression F1 Score: {}'.format(model2_f1))
print('logistic Regression Accuracy: {}'.format(model2_acc))

In [ ]:
# Set up the parameter grid
model3_paramGrid = ParamGridBuilder() \
    .addGrid() \
    .build()

# Treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
model3_crossval = CrossValidator(estimator=model3_pipeline,
                          estimatorParamMaps=model3_paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName='f1'),
                          numFolds=10,
                          seed=314,
                          parallelism=7)
#fit lr_model
model3_model = model3_crossval.fit(trainingData)

#make predictions using lr
model3_preds = model3_model.transform(testData)

In [11]:
#test performance
model3_f1 = f1_evaluator.evaluate(model3_preds)
model3_acc = accuracy_evaluator.evaluate(model3_preds)

print('logistic Regression F1 Score: {}'.format(model3_f1))
print('logistic Regression Accuracy: {}'.format(model3_acc))

0.7570295194733547

The RF classifier did reasonably well. In the future we will streamline the data pipeline process, evaluate more metrics, and vary RF parameters. We will also collect data from other years. This should be simple to do, and it will be useful since we had to drop so many duplicates and nulls.